In [1]:
import pandas as pd
import numpy as np

In [31]:
cluster = "pleiades"

In [32]:
df = pd.read_csv(f"{cluster}_luminosities.csv")

In [33]:
extra_campaigns = {"ngc2682": 5, "ngc2168": 0}

In [34]:
df.columns.values


array(['Unnamed: 0', 'EPIC', 'todrop', 'Dataset Name', 'Campaign',
       'Data Release', 'RAJ2000_K2', 'DEJ2000_K2', 'Target Type', 'RA PM',
       'RA PM Err', 'Dec PM', 'Dec PM Err', 'Plx', 'Plx Err', 'U Mag',
       'U Mag Err', 'B Mag', 'B Mag Err', 'V Mag', 'V Mag Err', 'G Mag',
       'G Mag Err', 'R Mag', 'R Mag Err', 'I Mag', 'I Mag Err', 'Z Mag',
       'Z Mag Err', 'J Mag', 'J Mag Err', 'H Mag', 'H Mag Err', 'K Mag',
       'K Mag Err', 'KEP Mag', 'Kep Flag', 'Hip ID', 'Tyc ID', 'SDSS ID',
       'UCAC ID', '2MASS ID', '2MASS Flag', 'crowding', 'contamination',
       'flux fraction', 'cdpp3', 'cdpp6', 'cdpp12', 'Module', 'Output',
       'Channel', 'Nearest Neighbor', 'Nomad ID', 'dist_mean_K2_arcmin',
       'RAJ2000_mean', 'DEJ2000_mean', 'RAJ2000_Olivares',
       'DEJ2000_Olivares', 'PMem_Olivares', 'binary_Olivares',
       'gaia_id_GaiaC', 'RAJ2000_GaiaC', 'DEJ2000_GaiaC', 'PMem_GaiaC',
       'dist_Olivares_GaiaC', 'RAJ2000_Olivares_GaiaC',
       'DEJ2000_Olivares_G

In [35]:

df[["EPIC", "PMem_mean"]]

,EPIC,PMem_mean
0,211091768.0,0.949999
1,211091768.0,0.949999
2,211008578.0,0.949998
3,211029232.0,0.949996
4,211029232.0,0.949996
...,...,...
900,211088007.0,0.900000
901,211088007.0,0.900000
902,211085527.0,0.900000
903,211040918.0,0.900000


In [43]:
df.EPIC = df.EPIC.astype(int)
df.Rstar = np.round(df.Rstar,2)
df.e_Rstar = np.round(df.e_Rstar,2).fillna(-999)
df.PMem_mean = np.round(df.PMem_mean,2).fillna(-999)
df.Lum_Kepler = df.Lum_Kepler.fillna(-999)
df.e_Lum_Kepler = df.e_Lum_Kepler.fillna(-999)
df.Campaign = np.rint(df.Campaign).fillna(extra_campaigns.setdefault(cluster, -999)).astype(int)
df.Teff_median = np.rint(df.Teff_median).fillna(-999).astype(int)
df.Teff_std = np.rint(df.Teff_std).fillna(-999).astype(int)
df.todrop = df.todrop.fillna("").str.replace("outlier","")
dfr = df.rename(index=str, columns={"EPIC":"ID",
                                   "todrop":"outlier",
                                   "Teff_median":"Teff",
                                   "Teff_std":"e_Teff",
                                   "Rstar":"R",
                                   "e_Rstar":"e_R",
                                   "PMem_mean":"pmem",
                                    "RAJ2000_K2":"RAJ2000",
                                    "DEJ2000_K2":"DEJ2000",
                                   })

In [46]:
dfr[["ID","outlier", "Teff", "e_Teff",
     "R", "e_R", "Lum_Kepler", "e_Lum_Kepler",
     "pmem","Campaign", "RAJ2000", "DEJ2000"]][dfr.ID == 211110493]

,ID,outlier,Teff,e_Teff,R,e_R,Lum_Kepler,e_Lum_Kepler,pmem,Campaign,RAJ2000,DEJ2000
155,211110493,,-999,-999,NaN,-999.0,-999.0,-999.0,1.0,4,55.75068,24.731255
